# Assignment 2

Group 8 - Hyungue Lim and Aaron Spaulding

We would like to implement a SVM on the bankruptcy dataset

In [1]:
import copy
import random 

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest

from matplotlib import pyplot as plt

import cvxopt as opt
from cvxopt import matrix, solvers

In [2]:
random.seed(1)

data_path = r'financial_datamining/data/bankruptcy.csv'

# Dataset
1) Size
    - Sales
    
2) Profit
    - ROCE: profit before tax=capital employed (%)
    - FFTL: funds flow (earnings before interest, tax & depreciation)=total liabilities
3) Gearing
    - GEAR: (current liabilities + long-term debt)=total assets
    - CLTA: current liabilities=total assets
4) Liquidity
    - CACL: current assets=current liabilities
    - QACL: (current assets – stock)=current liabilities
    - WCTA: (current assets – current liabilities)=total assets
5) LAG: number of days between account year end and the date the annual report and accounts
were filed at company registry.
6) AGE: number of years the company has been operating since incorporation date.
7) CHAUD: coded 1 if changed auditor in previous three years, 0 otherwise
8) BIG6: coded 1 if company auditor is a Big6 auditor, 0 otherwise
9) FAIL: The target variable, either = 1 or 0.

In [3]:
bankruptcy_df = pd.read_csv(data_path)

X = bankruptcy_df[bankruptcy_df.columns.difference(['Firm', 'FAIL'])]
y = bankruptcy_df[['FAIL']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.feature_selection import SelectKBest
k = 10
prep = SelectKBest(k=k)
X_train = prep.fit_transform(X_train, y_train.values.flatten())
X_test = prep.fit_transform(X_test, y_test.values.flatten())

## Kernal

### Linear Kernal
$$X*X^T$$

### Polynomial Kernal
$$K(x^{(i)}, c^{(j)}) = (x^{(i)^T} x^{(j)} + 1)^d$$

### Gaussian Kernal
$$K(x^{(i)}, x^{(j)}) = exp(\frac{-||x^{(i)}-x^{(j)}||^2}{2\sigma^2})$$

In [4]:
def kernel(XTest, XTrain, ktype = 0, sigma = 0):
    # Write your code below
    """
    ktype=0 : RBF
    ktype=1 : Polynomial (sigma is the exponent)
    ktype=2 : Linear
    """

    m = XTest.shape[0]
    if ktype == 0:
        
        K = np.zeros((m,m))
        for i in range(m):
            if m > 1:
                K[i] = np.exp((-np.linalg.norm(XTest[i] - XTrain, axis=1)**2) / (2*(sigma**2)))
            elif m == 1:
                K = np.exp((-np.linalg.norm(XTest[i] - XTrain, axis=1)**2) / (2*(sigma**2)))
    elif ktype == 1:
        K = (1 + XTest.dot(XTrain.T))**sigma

    elif ktype == 2:
        K = XTest.dot(XTrain.T)

    return K   

## Intercept

$$b=\frac{1}{N_s}(\sum_{i=1}^{N_s}(y_i-\sum_{j=1}^{N_s}a_iy_iK(x^{(i)},x^{(j)})))$$

In [5]:
def intercept(XTest, XTrain, yTrain, alpha):
    global new_kernel, ktype, sigma
    
    support_vector = alpha > 0.0001
    t_train = yTrain.copy()
    t_train[yTrain == 0] = -1
    t_train[yTrain == 1] = 1
    t_train = t_train

    train_kernel = X_train[support_vector.reshape(len(support_vector), )]
    new_kernel = kernel(train_kernel[[0]], train_kernel, ktype, sigma)
    if len(new_kernel.shape) == 2:
        num_bs = new_kernel.shape[1]
    elif len(new_kernel.shape) == 1:
        num_bs = new_kernel.shape[0]
        
    b = np.zeros(num_bs)
    for i in range(num_bs):
        b[i] = t_train[i] - (alpha[support_vector, None] * t_train[support_vector] * kernel(train_kernel[[i]], train_kernel, ktype, sigma).reshape(num_bs, -1)).sum()  


    return b.mean()

## Predict

$$g(x)=\sum_{j=1}^{N} \alpha_j y_j K(x,x^{(j)}) + b$$

In [6]:
def predict(XTest, XTrain, yTrain, alpha):
    global new_kernel, ktype, sigma
    
    b = intercept(XTest, XTrain, yTrain, alpha)
    support_vector = alpha > 0.0001
    t_train = yTrain.copy()
    t_train[yTrain == 0] = -1
    t_train[yTrain == 1] = 1
    t_train = t_train

    train_kernel = X_train[support_vector.reshape(len(support_vector), )]
    if len(new_kernel.shape) == 2:
        num_bs = new_kernel.shape[1]
    elif len(new_kernel.shape) == 1:
        num_bs = new_kernel.shape[0]
    result = np.zeros(len(XTest))
    
    for i in range(len(XTest)):
        result[i] = (alpha[support_vector, None] * t_train[support_vector] * kernel(X_test[[i]], train_kernel, ktype, sigma).reshape(num_bs, -1)).sum() + b 
    
    pred = result > 0
    return pred

In [7]:
# global variables
# ktype: 0 = rbf, 1 = polynomial, 2 = linear
# sigma: if rbf, sigma = sigma; if polynomial, sigma = polynomial degree

new_kernel = []
ktype = 2
sigma = 2


# Linear Kernel
TrainSize = X_train.shape[0]
K = kernel(X_train, X_train, ktype, sigma)

ymat = np.diag(y_train.values.flatten())
m = X_train.shape[0]
P = opt.matrix(np.outer(y_train, y_train) * K)
q = opt.matrix(-np.ones((m, 1)))
G = opt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
h = opt.matrix(np.hstack((np.zeros(m), np.ones(m) * 1)))
A = opt.matrix(y_train.values, (1, m), "d")
b = opt.matrix(np.zeros(1))
opt.solvers.options["show_progress"] = False
sol = opt.solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol["x"])
(alphas > 0.0001).sum()



pred = predict(X_test, X_train, y_train.values, alphas)
print('Accuracy: %f\n' % (np.mean(pred == np.array(y_test).reshape(-1)) * 100))

Accuracy: 100.000000

